## 1. Import Libraries

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import xgboost as xgb
import joblib
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

## 2. Data Loading and Preprocessing

In [17]:
def load_and_preprocess_data(file_path):
    print("\n" + "="*80)
    print("📊 LOADING AND PREPROCESSING DATA")
    print("="*80)
    
    df = pd.read_csv("Water Quality Prediction.csv")
    
    # Check for missing values
    missing_values = df.isnull().sum()
    print("\n📋 Missing values before imputation:")
    display(missing_values[missing_values > 0])
    
    # Display dataset info
    print(f"\n📈 Dataset shape: {df.shape[0]} rows, {df.shape[1]} columns")
    
    # Display sample
    print("\n🔍 Sample of the dataset:")
    display(df.head())
    
    return df
load_and_preprocess_data("Water Quality Prediction.csv")


📊 LOADING AND PREPROCESSING DATA

📋 Missing values before imputation:


pH                        20231
Iron                       6991
Nitrate                   18695
Chloride                  30834
Lead                       4684
Zinc                      27675
Color                       981
Turbidity                  8694
Fluoride                  33218
Copper                    34882
Odor                      31332
Sulfate                   34525
Conductivity              28803
Chlorine                  10162
Manganese                 19339
Total Dissolved Solids      298
Source                    15535
Water Temperature         29688
Air Temperature            5303
Month                     16921
Day                       17549
Time of Day               20361
dtype: int64


📈 Dataset shape: 1048575 rows, 24 columns

🔍 Sample of the dataset:


,Index,pH,Iron,Nitrate,Chloride,Lead,Zinc,Color,Turbidity,Fluoride,...,Chlorine,Manganese,Total Dissolved Solids,Source,Water Temperature,Air Temperature,Month,Day,Time of Day,Target
0,0,8.332988,0.000083,8.605777,122.799772,3.710000e-52,3.434827,Colorless,0.022683,0.607283,...,3.708178,2.270000e-15,332.118789,NaN,NaN,43.493324,January,29.0,4.0,0
1,1,6.917863,0.000081,3.734167,227.029851,7.850000e-94,1.245317,Faint Yellow,0.019007,0.622874,...,3.292038,8.020000e-07,284.641984,Lake,15.348981,71.220586,November,26.0,16.0,0
2,2,5.443762,0.020106,3.816994,230.995630,5.290000e-76,0.528280,Light Yellow,0.319956,0.423423,...,3.560224,7.007989e-02,570.054094,River,11.643467,44.891330,January,31.0,8.0,0
3,3,7.955339,0.143988,8.224944,178.129940,4.000000e-176,4.027879,Near Colorless,0.166319,0.208454,...,3.516907,2.468295e-02,100.043838,Ground,10.092392,60.843233,April,1.0,21.0,0
4,4,8.091909,0.002167,9.925788,186.540872,4.170000e-132,3.807511,Light Yellow,0.004867,0.222912,...,3.177849,3.296139e-03,168.075545,Spring,15.249416,69.336671,June,29.0,7.0,0


,Index,pH,Iron,Nitrate,Chloride,Lead,Zinc,Color,Turbidity,Fluoride,...,Chlorine,Manganese,Total Dissolved Solids,Source,Water Temperature,Air Temperature,Month,Day,Time of Day,Target
0,0,8.332988,8.350000e-05,8.605777,122.799772,3.710000e-52,3.434827,Colorless,0.022683,0.607283,...,3.708178,2.270000e-15,332.118789,NaN,NaN,43.493324,January,29.0,4.0,0
1,1,6.917863,8.050000e-05,3.734167,227.029851,7.850000e-94,1.245317,Faint Yellow,0.019007,0.622874,...,3.292038,8.020000e-07,284.641984,Lake,15.348981,71.220586,November,26.0,16.0,0
2,2,5.443762,2.010586e-02,3.816994,230.995630,5.290000e-76,0.528280,Light Yellow,0.319956,0.423423,...,3.560224,7.007989e-02,570.054094,River,11.643467,44.891330,January,31.0,8.0,0
3,3,7.955339,1.439878e-01,8.224944,178.129940,4.000000e-176,4.027879,Near Colorless,0.166319,0.208454,...,3.516907,2.468295e-02,100.043838,Ground,10.092392,60.843233,April,1.0,21.0,0
4,4,8.091909,2.167128e-03,9.925788,186.540872,4.170000e-132,3.807511,Light Yellow,0.004867,0.222912,...,3.177849,3.296139e-03,168.075545,Spring,15.249416,69.336671,June,29.0,7.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,1048570,8.186428,1.977157e-02,5.968850,115.963429,1.590000e-113,2.277221,Near Colorless,0.020610,1.194165,...,2.718928,2.603963e-02,220.571485,Stream,5.491908,43.817452,March,15.0,14.0,0
1048571,1048571,8.046225,1.160000e-05,3.678714,148.053168,5.930000e-25,0.483369,Near Colorless,0.878835,1.203689,...,2.312058,4.427830e-02,406.095969,Lake,10.143768,34.626853,December,16.0,0.0,0
1048572,1048572,7.443582,3.103077e-03,7.893399,174.677900,1.080000e-15,1.461659,Near Colorless,0.701053,0.115412,...,3.410297,2.350000e-05,439.086461,Spring,14.612881,55.460415,June,18.0,16.0,0
1048573,1048573,6.897232,6.980000e-10,5.757980,125.564223,1.300000e-18,0.804589,Near Colorless,0.156424,0.554729,...,2.803234,3.220000e-19,278.051032,Well,7.351956,65.055864,May,5.0,3.0,0


## 3. Feature Engineering

In [20]:
def engineer_features(df):
    print("\n" + "="*80)
    print("🔧 FEATURE ENGINEERING")
    print("="*80)
    
    # Create a copy to avoid SettingWithCopyWarning
    df_processed = df.copy()
    
    # Create Water Quality Index if not present
    if 'WQI' not in df_processed.columns:
        print("\n🧪 Creating Water Quality Index (WQI)")
        
        # Ensure all required columns exist and handle missing values
        required_cols = ['pH', 'Chlorine', 'Turbidity', 'Lead', 'Manganese', 'Fluoride', 'Iron', 'Nitrate']
        for col in required_cols:
            if col not in df_processed.columns:
                print(f"   - Column {col} not found. Creating with placeholder values.")
                df_processed[col] = df_processed[required_cols[0]].mean() if required_cols[0] in df_processed.columns else 0
        
        # Create WQI with bounds checking to avoid outliers
        df_processed['pH_norm'] = df_processed['pH'].clip(6, 9) / 9  # Normalize pH to 0-1 range with clipping
        df_processed['Turbidity_norm'] = 1 - (df_processed['Turbidity'] / df_processed['Turbidity'].quantile(0.95)).clip(0, 1)
        df_processed['Lead_norm'] = 1 - (df_processed['Lead'] / df_processed['Lead'].quantile(0.95)).clip(0, 1)
        
        df_processed['WQI'] = (
            df_processed['pH_norm'] * 0.15 + 
            df_processed['Chlorine'].clip(0, df_processed['Chlorine'].quantile(0.95)) * 0.12 + 
            df_processed['Turbidity_norm'] * 0.15 + 
            df_processed['Lead_norm'] * 0.2 +
            (1 - df_processed['Manganese'] / df_processed['Manganese'].quantile(0.95)).clip(0, 1) * 0.1 + 
            df_processed['Fluoride'].clip(0, df_processed['Fluoride'].quantile(0.95)) * 0.08 +
            (1 - df_processed['Iron'] / df_processed['Iron'].quantile(0.95)).clip(0, 1) * 0.1 + 
            (1 - df_processed['Nitrate'] / df_processed['Nitrate'].quantile(0.95)).clip(0, 1) * 0.1
        )
        
        # Normalize WQI to 0-1 range
        df_processed['WQI'] = df_processed['WQI'].clip(0, 1)
        print("   ✓ WQI created successfully")
    
    # Add California cities with realistic distribution
    if 'CA_City' not in df_processed.columns:
        print("\n🏙️ Adding California cities")
        ca_cities = [
            'Los Angeles', 'San Francisco', 'San Diego', 'Sacramento', 'Fresno',
            'San Jose', 'Oakland', 'Long Beach', 'Bakersfield', 'Anaheim',
            'Santa Ana', 'Riverside', 'Stockton', 'Irvine', 'Chula Vista',
            'Fremont', 'Santa Clarita', 'San Bernardino', 'Modesto', 'Fontana'
        ]
        
        # Population-weighted distribution (larger cities appear more frequently)
        city_weights = [0.15, 0.12, 0.1, 0.08, 0.06, 0.07, 0.05, 0.05, 0.04, 0.04,
                       0.03, 0.03, 0.03, 0.03, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02]
        
        df_processed['CA_City'] = np.random.choice(ca_cities, size=len(df_processed), p=city_weights)
        print("   ✓ Added California cities")
    
    # Add pipeline materials with realistic distribution
    if 'Pipeline_Material' not in df_processed.columns:
        print("\n🔧 Adding pipeline materials")
        materials = ['PVC', 'Copper', 'Iron', 'Concrete', 'HDPE']
        material_weights = [0.4, 0.25, 0.2, 0.1, 0.05]  # More realistic distribution
        
        df_processed['Pipeline_Material'] = np.random.choice(materials, size=len(df_processed), p=material_weights)
        print("   ✓ Added pipeline materials")
    
    # Add pipeline age with realistic distribution
    if 'Pipeline_Age' not in df_processed.columns:
        print("\n⏱️ Adding pipeline age")
        # Most pipes are newer, with fewer very old pipes
        df_processed['Pipeline_Age'] = np.random.gamma(shape=2.0, scale=10.0, size=len(df_processed)).astype(int).clip(1, 75)
        print("   ✓ Added pipeline age")
    
    # Add treatment facility type
    if 'Treatment_Facility' not in df_processed.columns:
        print("\n🏭 Adding treatment facility types")
        facilities = ['Conventional', 'Membrane', 'UV Disinfection', 'Reverse Osmosis', 'Chlorination']
        facility_weights = [0.35, 0.25, 0.2, 0.15, 0.05]
        
        df_processed['Treatment_Facility'] = np.random.choice(facilities, size=len(df_processed), p=facility_weights)
        print("   ✓ Added treatment facility types")
    
    # Create seasonal indicator from Month
    if 'Month' in df_processed.columns and 'Season' not in df_processed.columns:
        print("\n🌡️ Creating seasonal indicators")
        season_map = {
            1: 'Winter', 2: 'Winter', 3: 'Spring', 4: 'Spring',
            5: 'Spring', 6: 'Summer', 7: 'Summer', 8: 'Summer',
            9: 'Fall', 10: 'Fall', 11: 'Fall', 12: 'Winter'
        }
        df_processed['Season'] = df_processed['Month'].map(season_map)
        print("   ✓ Added seasonal indicators")
    
    # Create target variable (water quality percentage)
    if 'Water_Quality_Pct' not in df_processed.columns:
        print("\n🎯 Creating target variable (Water Quality Percentage)")
        df_processed['Water_Quality_Pct'] = df_processed['WQI'] * 100
        print("   ✓ Created water quality percentage target")
    
    # Check for and remove extreme outliers in numerical columns
    print("\n🔍 Handling outliers in numerical columns")
    for col in df_processed.select_dtypes(include=['float64', 'int64']).columns:
        if col not in ['Water_Quality_Pct', 'WQI', 'Pipeline_Age']:  # Skip target and engineered features
            q1 = df_processed[col].quantile(0.01)
            q3 = df_processed[col].quantile(0.99)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            
            # Count outliers before clipping
            outliers_count = ((df_processed[col] < lower_bound) | (df_processed[col] > upper_bound)).sum()
            if outliers_count > 0:
                print(f"   - Clipped {outliers_count} outliers in {col}")
                
            df_processed[col] = df_processed[col].clip(lower_bound, upper_bound)
    
    print("\n✅ Feature engineering complete")
    
    # Display the engineered features
    print("\n🔍 Sample of engineered dataset:")
    display(df_processed.head())
    
    return df_processed
print(engineer_features)

<function engineer_features at 0x1275f8b80>


## 4. Data Preparation for Modeling

In [5]:
def prepare_for_modeling(df):
    print("\n" + "="*80)
    print("🔄 PREPARING DATA FOR MODELING")
    print("="*80)
    
    # Identify numerical and categorical columns
    numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    
    # Remove target from features if present
    if 'Water_Quality_Pct' in numerical_cols:
        numerical_cols.remove('Water_Quality_Pct')
        print("\n🎯 Removed target variable from features")
    
    print(f"\n📊 Using {len(numerical_cols)} numerical features:")
    print(f"   {', '.join(numerical_cols)}")
    
    print(f"\n📋 Using {len(categorical_cols)} categorical features:")
    print(f"   {', '.join(categorical_cols)}")
    
    # Create preprocessing pipelines
    print("\n⚙️ Creating preprocessing pipelines")
    numeric_transformer = Pipeline(steps=[
        ('imputer', KNNImputer(n_neighbors=5)),
        ('scaler', MinMaxScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Combine preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])
    
    # Split data
    print("\n✂️ Splitting data into train and test sets")
    X = df.drop(['Water_Quality_Pct'], axis=1)
    y = df['Water_Quality_Pct']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"   - Training set: {X_train.shape[0]} samples")
    print(f"   - Test set: {X_test.shape[0]} samples")
    
    return X_train, X_test, y_train, y_test, preprocessor, numerical_cols, categorical_cols


## 5. Model Training and Evaluation

In [6]:
def train_and_evaluate_models(X_train, X_test, y_train, y_test, preprocessor):
    print("\n" + "="*80)
    print("🧠 TRAINING AND EVALUATING MODELS")
    print("="*80)
    
    # Random Forest with hyperparameter tuning
    print("\n🌲 Training Random Forest model")
    rf_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(random_state=42))
    ])
    
    rf_params = {
        'regressor__n_estimators': [100, 200, 300],
        'regressor__max_depth': [10, 20, 30, None],
        'regressor__min_samples_split': [2, 5, 10],
        'regressor__min_samples_leaf': [1, 2, 4]
    }
    
    print("   - Tuning Random Forest hyperparameters...")
    rf_search = RandomizedSearchCV(rf_pipeline, rf_params, n_iter=10, cv=3, n_jobs=-1, random_state=42, verbose=0)
    rf_search.fit(X_train, y_train)
    print(f"   ✓ Best parameters: {rf_search.best_params_}")
    
    # XGBoost with hyperparameter tuning
    print("\n🚀 Training XGBoost model")
    xgb_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', xgb.XGBRegressor(random_state=42))
    ])
    
    xgb_params = {
        'regressor__n_estimators': [100, 200, 300],
        'regressor__max_depth': [3, 5, 7, 9],
        'regressor__learning_rate': [0.01, 0.05, 0.1, 0.2],
        'regressor__subsample': [0.8, 0.9, 1.0],
        'regressor__colsample_bytree': [0.8, 0.9, 1.0]
    }
    
    print("   - Tuning XGBoost hyperparameters...")
    xgb_search = RandomizedSearchCV(xgb_pipeline, xgb_params, n_iter=10, cv=3, n_jobs=-1, random_state=42, verbose=0)
    xgb_search.fit(X_train, y_train)
    print(f"   ✓ Best parameters: {xgb_search.best_params_}")
    
    # Gradient Boosting
    print("\n🔥 Training Gradient Boosting model")
    gb_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', GradientBoostingRegressor(random_state=42))
    ])
    
    gb_params = {
        'regressor__n_estimators': [100, 200, 300],
        'regressor__max_depth': [3, 5, 7],
        'regressor__learning_rate': [0.01, 0.05, 0.1, 0.2],
        'regressor__subsample': [0.8, 0.9, 1.0]
    }
    
    print("   - Tuning Gradient Boosting hyperparameters...")
    gb_search = RandomizedSearchCV(gb_pipeline, gb_params, n_iter=10, cv=3, n_jobs=-1, random_state=42, verbose=0)
    gb_search.fit(X_train, y_train)
    print(f"   ✓ Best parameters: {gb_search.best_params_}")
    
    # Evaluate models
    models = {
        'Random Forest': rf_search.best_estimator_,
        'XGBoost': xgb_search.best_estimator_,
        'Gradient Boosting': gb_search.best_estimator_
    }
    
    results = {}
    
    print("\n📊 MODEL EVALUATION RESULTS")
    print("-"*50)
    
    for name, model in models.items():
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results[name] = {
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae,
            'R²': r2
        }
        
        print(f"\n📈 {name} Results:")
        print(f"   - MSE: {mse:.4f}")
        print(f"   - RMSE: {rmse:.4f}")
        print(f"   - MAE: {mae:.4f}")
        print(f"   - R²: {r2:.4f}")
    
    # Find best model
    best_model_name = max(results, key=lambda x: results[x]['R²'])
    best_model = models[best_model_name]
    
    print("\n🏆 BEST MODEL")
    print(f"   - {best_model_name} with R² = {results[best_model_name]['R²']:.4f}")
    
    # Save best model
    joblib.dump(best_model, 'water_quality_model.pkl')
    print("\n💾 Best model saved as 'water_quality_model.pkl'")
    
    # Visualize model comparison
    plt.figure(figsize=(12, 6))
    model_names = list(results.keys())
    r2_values = [results[name]['R²'] for name in model_names]
    
    plt.bar(model_names, r2_values, color=['#3498db', '#2ecc71', '#e74c3c'])
    plt.title('Model Comparison - R² Score', fontsize=15)
    plt.ylabel('R² Score', fontsize=12)
    plt.ylim(0, 1)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Add value labels on top of bars
    for i, v in enumerate(r2_values):
        plt.text(i, v + 0.01, f'{v:.4f}', ha='center', fontsize=12)
        
    plt.tight_layout()
    plt.show()
    
    return best_model, models, results


## 6. Manual Input Prediction Function

In [8]:
def predict_water_quality(model, numerical_cols, categorical_cols):
    print("\n" + "="*80)
    print("🔮 WATER QUALITY PREDICTION TOOL")
    print("="*80)
    print("\nEnter values for the following parameters:")
    
    # Create empty dictionary to store inputs
    input_data = {}
    
    # Get numerical inputs
    print("\n📊 NUMERICAL PARAMETERS")
    print("-"*50)
    for col in numerical_cols:
        if col not in ['WQI', 'Water_Quality_Pct', 'pH_norm', 'Turbidity


SyntaxError: unterminated string literal (detected at line 14) (316743053.py, line 14)

In [9]:
# Main execution for inference
def main():
    try:
        # Load the saved model
        print("\n" + "="*80)
        print("🔄 LOADING WATER QUALITY PREDICTION MODEL")
        print("="*80)
        
        try:
            model = joblib.load('water_quality_model.pkl')
            print("✅ Model loaded successfully!")
        except FileNotFoundError:
            print("❌ Model file not found. Please train the model first.")
            return
        
        # Define the columns needed for prediction
        numerical_cols = [
            'pH', 'Chlorine', 'Turbidity', 'Lead', 'Manganese', 
            'Fluoride', 'Iron', 'Nitrate', 'Pipeline_Age'
        ]
        
        categorical_cols = [
            'CA_City', 'Pipeline_Material', 'Treatment_Facility', 'Season', 'Source'
        ]
        
        # Run prediction
        predict_water_quality(model, numerical_cols, categorical_cols)
        
    except Exception as e:
        print(f"❌ An error occurred: {e}")

# Run the inference
if __name__ == "__main__":
    main()


🔄 LOADING WATER QUALITY PREDICTION MODEL
❌ Model file not found. Please train the model first.
